In [22]:
import csv
from collections import Counter
import pandas as pd
from itertools import chain, combinations
from tabulate import tabulate

In [23]:
# Read the .pb file 

path = "netherlands_amsterdam_166_.pb"  # set this variable
with open(path, 'r', newline='', encoding="utf-8") as csvfile:
    meta = {}
    projects = {}
    votes = {}
    section = ""
    header = []
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if str(row[0]).strip().lower() in ["meta", "projects", "votes"]:
            section = str(row[0]).strip().lower()
            header = next(reader)
        elif section == "meta":
            meta[row[0]] = row[1].strip()
        elif section == "projects":
            projects[row[0]] = {}
            for it, key in enumerate(header[1:]):
                projects[row[0]][key.strip()] = row[it+1].strip()
        elif section == "votes":
            votes[row[0]] = {}
            for it, key in enumerate(header[1:]):
                votes[row[0]][key.strip()] = row[it+1].strip()

In [24]:
#Extract required data from the file
C=set(projects.keys())
V=set(votes.keys())
n=len(V)
approval_dict = {key: set(value['vote'].split(',')) for key, value in votes.items()}

In [25]:
#Creating a df with the projects and the number of votes they got
all_values = set()
for value_set in approval_dict.values():
    all_values.update(value_set)

# Count occurrences of each value in the entire dictionary
count_dict = Counter()
for value_set in approval_dict.values():
    count_dict.update(value_set)

# Create a list with unique values and their counts
result_list = [[value, count_dict[value]] for value in all_values]

greedy_list = pd.DataFrame(result_list, columns=['Value', 'Count'])
greedy_list = greedy_list.sort_values(by='Count', ascending=False)

In [26]:
# x = int(input("Enter the size of the Winners set (1 to 10): "))
# # Select the top x values from the sorted DataFrame
# top_x_values = greedy_list.head(x)['Value']

# # Create a Winners set from the top x values
# W = set(top_x_values)

# print("Winners set:", W)

In [27]:
# k=len(W)
# n=len(V)

In [28]:
# non_winners=C-W
# # 

In [29]:
# counts = {}
# breaks_jr=False

# for non_winner in non_winners: # iterate through non winners (for all c belonging to C)
#     counts[non_winner] = 0
#     for key, value in approval_dict.items(): # iterate through all approval lists 
#         if non_winner in value and not set(W).intersection(value): #if c belongs to approval list but W intersection Approval list is null
#             counts[non_winner] += 1
#             if counts[non_winner] >= n/k:
#                 breaks_jr = True
#                 break  # Exit the inner loop if the condition is met

# if breaks_jr:
#     print("Winner set breaks JR.")
# else:
#     print("Winner set satisfy JR.")

In [30]:
# from itertools import chain, combinations

# #produces possible subsets of all candidates
# def powerset(iterable, max_size): 
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(1, min(len(s)+1, max_size+1)))

# def election_algorithm(approval_lists, winners, k, n, C):
#     S = set(s for s in powerset(C, k) if not set(s).issubset(winners)) #create all subsets of size <=k

#     for s in S:#iterate through all subsets
#         l = len(s)
#         count = 0
#         for approval_list in approval_lists.values(): #iterate through all approval_lists/profile
#             if set(s).issubset(approval_list): #if s is a subset of the profile
#                 if abs(len(set(approval_list) & set(winners))) < l: #if profile intersection W is < l
#                     count += 1
#         if count >= l * n / k:
#             print("breaks EJR")
#             return

#     print("Satisfies EJR")


# election_algorithm(approval_dict, W, k, n, C)


In [ ]:
# Define the function to create the powerset
def powerset(iterable, max_size): 
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, min(len(s)+1, max_size+1)))

# Define the function to create the set W dynamically based on its size
def create_set_W(df, W):
    return set(df.head(W)['Value'])

# Define the function for JR
def JR(non_winners, approval_dict, W_set, k, n):
    counts = {}
    breaks_jr = False

    for non_winner in non_winners:
        counts[non_winner] = 0
        for key, value in approval_dict.items():
            if non_winner in value and not set(W_set).intersection(value):
                counts[non_winner] += 1
                if counts[non_winner] >= n / k:
                    breaks_jr = True
                    break
        if breaks_jr:
            return "Breaks JR."
        
    return "Satisfies JR."

# Define the function for EJR
def EJR(approval_lists, winners, k, n, C):
    S = set(s for s in powerset(C, k) if not set(s).issubset(winners)) #create all subsets of size <=k

    for s in S:#iterate through all subsets
        l = len(s)
        count = 0
        for approval_list in approval_lists.values(): #iterate through all approval_lists/profile
            if set(s).issubset(approval_list): #if s is a subset of the profile
                if abs(len(set(approval_list) & set(winners))) < l: #if profile intersection W is < l
                    count += 1
        if count >= l * n / k:
            return "breaks EJR"
            
    return "Satisfies EJR"


results = []

# Iterate over different values of W
for W_value in range(1, 11):
    W_set = create_set_W(greedy_list, W_value)
    k = len(W_set)
    non_winners = C - W_set
    
    # Check for JR
    result1 = JR(non_winners, approval_dict, W_set, k, n)
    
    # Check for EJR
    result2 = EJR(approval_dict, W_set, k, n, C)
    #print(result2)
    results.append((W_value, result1, result2))

    results_df = pd.DataFrame(results, columns=['W', 'JR', 'EJR'])

    print(tabulate(results_df, headers='keys', tablefmt='pretty'))



+---+---+---------------+---------------+
|   | W |      JR       |      EJR      |
+---+---+---------------+---------------+
| 0 | 1 | Satisfies JR. | Satisfies EJR |
+---+---+---------------+---------------+
+---+---+---------------+---------------+
|   | W |      JR       |      EJR      |
+---+---+---------------+---------------+
| 0 | 1 | Satisfies JR. | Satisfies EJR |
| 1 | 2 | Satisfies JR. | Satisfies EJR |
+---+---+---------------+---------------+
+---+---+---------------+---------------+
|   | W |      JR       |      EJR      |
+---+---+---------------+---------------+
| 0 | 1 | Satisfies JR. | Satisfies EJR |
| 1 | 2 | Satisfies JR. | Satisfies EJR |
| 2 | 3 | Satisfies JR. | Satisfies EJR |
+---+---+---------------+---------------+
